In [28]:
import pandas as pd
import altair as alt
import os 
from vega_datasets import data

In [20]:
cuisine_mapping = pd.read_csv('data/Cuisinne Mapping.csv')

In [21]:
# List files in data/Taste/
files = os.listdir('data/Taste/')
# Read in all files with pandas
# Filename as a column
dfs = []
for file in files:
    dfs.append(pd.read_csv('data/Taste/' + file))
    
# Concatenate all files
taste = pd.concat(dfs)

In [54]:
# Merge with cuisine mapping
mapping = taste.merge(cuisine_mapping, on='cuisine')
# Filter out those with blank "ISO 31661 Numeric"
mapping_filtered = mapping[mapping['ISO 31661 Numeric'].notnull()]
mapping_filtered.loc[:,'id'] = mapping_filtered['ISO 31661 Numeric'].astype(int).astype(str)

/tmp/ipykernel_13713/816558526.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_filtered.loc[:,'id'] = mapping_filtered['ISO 31661 Numeric'].astype(int).astype(str)


In [55]:
mapping_filtered.head()

,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness,id,title,cuisine,Country,ISO 31661 Numeric,Notes
0,22.09,100.00,9.92,11.24,24.96,72.54,0.0,826,Real English Fish and Chips With Yorkshire Bee...,British,United Kingdom of Great Britain and Northern I...,826.0,NaN
1,32.53,49.92,10.51,6.23,29.49,100.00,0.0,826,Classic Fish and Chips,British,United Kingdom of Great Britain and Northern I...,826.0,NaN
2,24.30,100.00,14.88,39.97,25.74,75.86,534900.0,826,Winter Pudding Bombe,British,United Kingdom of Great Britain and Northern I...,826.0,NaN
3,84.42,57.29,5.43,2.79,26.08,100.00,0.0,826,Beer Battered Fish and Chips,British,United Kingdom of Great Britain and Northern I...,826.0,NaN
4,78.91,22.95,7.01,23.33,21.29,100.00,0.0,826,Sheet Pan Fish and Chips,British,United Kingdom of Great Britain and Northern I...,826.0,NaN


In [58]:
source = alt.topo_feature(data.world_110m.url, 'countries')
sphere = alt.sphere()
graticule = alt.graticule()

# flavors = mapping_filtered.columns[0:7]
# input_dropdown = alt.binding_select(options=flavors, name='Flavor')
# selection = alt.selection_single(fields=['Origin'], bind=input_dropdown)

flavor = 'sweetness'

# Layering and configuring the components
alt.Chart(source).mark_geoshape(
    stroke='black'
    ).encode(
        color= flavor + ':Q'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(mapping_filtered, 'id', [flavor])
).project(
    'naturalEarth1'
).properties(width=600, height=400).configure_view(stroke=None)

alt.Chart(...)

In [75]:
# Same as above, but with a filter for the flavor

# Keep first 8 columns of mapping_filtered
mapping_filtered_flavor = mapping_filtered[mapping_filtered.columns[0:8]]


mapping_filtered_flavor_pivoted = pd.melt(mapping_filtered_flavor, id_vars='id', value_vars=mapping_filtered_flavor.columns[1:8])
# Melt with 'id' as key

In [88]:
# Take average of value by id, variabl
averaged = mapping_filtered_flavor_pivoted.groupby(['id','variable']).mean()
# Reset hierarchical index
averaged = averaged.reset_index()

In [113]:
averaged

,id,variable,value
0,826,bitterness,2.258812e+01
1,826,fattiness,7.167444e+01
2,826,saltiness,7.293714e+01
3,826,savoriness,3.275014e+01
4,826,sourness,2.441220e+01
5,826,spiciness,1.078856e+05
6,840,bitterness,3.420870e+01
7,840,fattiness,6.156614e+01
8,840,saltiness,6.560166e+01
9,840,savoriness,3.721540e+01


In [115]:
## NOT WORKING - needs debugged (only showing spiciness for some reason)
input_dropdown = alt.binding_select(options=list(mapping_filtered_flavor.columns[0:7]), name='Flavor')
selection = alt.selection_single(fields=['variable'], bind=input_dropdown)

# Create same map as above but with a filter for the flavor
# Layering and configuring the components
chart = alt.Chart(source).mark_geoshape(
    ).encode(
        color= 'value:Q',
        opacity = 'variable:N'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(averaged, 'id', ['value','variable'])
).project(
    'equirectangular'
).properties(width=600, height=400
).configure_view(stroke=None
).add_selection(
    selection
).transform_filter(selection)
chart

alt.Chart(...)